In [1]:
!pip install sentencepiece
!pip install tensorflow_addons

     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 5.2 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import tensorflow as tf
import json
import numpy as np
import sys
from tensorflow import keras
from sklearn.metrics import f1_score

BASE_PATH="/content/drive/MyDrive/Disertatie_V2"

sys.path.append(BASE_PATH)

# local imports
from repo import dataGenerator
from repo import trainer
from repo.bertLayer import BertLayer

import data_generators
import model_factory

MAX_SEQ_LENGTH=128
MAX_CHAR_LENGTH=10

DATASET="legal"


In [8]:
# for BERT model
generator_train = data_generators.DataGeneratorBertCRF(path_to_data=f"datasets_vanilla/{DATASET}/train",
max_seq_length=MAX_SEQ_LENGTH, path_to_bert='BERT', batch_size=64, base_path=BASE_PATH)

generator_dev = data_generators.DataGeneratorBertCRF(path_to_data=f"datasets_vanilla/{DATASET}/dev",
max_seq_length=MAX_SEQ_LENGTH, path_to_bert='BERT', batch_size=64, base_path=BASE_PATH)

generator_test = data_generators.DataGeneratorBertCRF(path_to_data=f"datasets_vanilla/{DATASET}/test",
max_seq_length=MAX_SEQ_LENGTH, path_to_bert='BERT', batch_size=1, base_path=BASE_PATH)

generator_dev.tag2idx = generator_train.tag2idx 
generator_dev.tags = generator_train.tags

generator_test.tag2idx = generator_train.tag2idx 
generator_test.tags = generator_train.tags

model = model_factory.build_BERT_CRF(generator_train.max_seq_length, len(generator_train.tags),
                                 base_path=BASE_PATH, bert_path='BERT')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:376: UserWarning: CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476
  return py_builtins.overload_of(f)(*args)


In [ ]:
# for LSTM_CHARS model
generator_train = data_generators.DataGeneratorLSTMCharsCRF(path_to_data=f"datasets_vanilla/{DATASET}/train",
max_seq_length=MAX_SEQ_LENGTH, max_char_len=MAX_CHAR_LENGTH, batch_size=64, base_path=BASE_PATH)

generator_dev = data_generators.DataGeneratorLSTMCharsCRF(path_to_data=f"datasets_vanilla/{DATASET}/dev",
max_seq_length=MAX_SEQ_LENGTH, max_char_len=MAX_CHAR_LENGTH, batch_size=64, base_path=BASE_PATH)

generator_test = data_generators.DataGeneratorLSTMCharsCRF(path_to_data=f"datasets_vanilla/{DATASET}/test",
max_seq_length=MAX_SEQ_LENGTH, max_char_len=MAX_CHAR_LENGTH, batch_size=1, base_path=BASE_PATH)

data_generators.copy_vocabulary(generator_train, generator_test)
data_generators.copy_vocabulary(generator_train, generator_dev)

model = model_factory.build_LSTM_Chars_CRF(generator_train.max_seq_length, generator_train.max_char_length, 
                                       len(generator_train.words), len(generator_train.chars), len(generator_train.tags))

In [9]:
es = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, min_delta=0.01)
model.fit(generator_train, validation_data=generator_dev, epochs=5, verbose=1, callbacks=[es])

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/text/crf.py:546: UserWarning: CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476
  "CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476"


Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:376: UserWarning: CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476
  return py_builtins.overload_of(f)(*args)


86/86 [==============================] - 3140s 36s/step - loss: 67.8541 - val_loss: 39.4959
Epoch 2/5
86/86 [==============================] - 3063s 36s/step - loss: 27.5812 - val_loss: 29.0948
Epoch 3/5
86/86 [==============================] - 2961s 34s/step - loss: 21.4009 - val_loss: 24.3780
Epoch 4/5
86/86 [==============================] - 2964s 35s/step - loss: 18.0183 - val_loss: 21.4732
Epoch 5/5
86/86 [==============================] - 2927s 34s/step - loss: 15.8509 - val_loss: 19.2813


In [10]:
def pred2labelCRF(pred, tag2idx):
        idx2tag = {idx : tag for tag, idx in tag2idx.items()}
        pred_raw = list(map(lambda x: idx2tag[x], pred))
        return pred_raw

In [ ]:
def pred2label(pred, tag2idx):
        out = []
        idx2tag = {idx : tag for tag, idx in tag2idx.items()}
        pred = np.reshape(pred, pred.shape[1:])
        pred_idxs = np.argmax(pred, axis=-1)
        pred_raw = list(map(lambda x: idx2tag[x], pred_idxs))
        return pred_raw

In [12]:
rev_func = pred2labelCRF

In [20]:
idxs = np.random.choice(list(range(len(generator_test))), 5)

for idx in idxs:
    gt = generator_test[idx][1]
    
    gt = gt.flatten()
    gt = rev_func(gt, generator_test.tag2idx)
    gt = list(filter(lambda x: x != data_generators.PADDING, gt))

    pred = model.predict(generator_test[idx][0])[0]
    pred = pred.flatten()
    pred = rev_func(pred, generator_test.tag2idx)
    pred = pred[:len(gt)]
    
    print(idx)
    print('gt   : ', gt)
    print('pred : ', pred)
    print()

548
gt   :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'B-TIME', 'I-TIME', 'I-TIME', 'O']
pred :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'I-PER', 'I-PER', 'I-LOC', 'I-LOC', 'O', 'B-TIME', 'I-TIME', 'I-TIME', 'O']

669
gt   :  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'I-LOC', 'O', 'B-TIME', 'I-TIME', 'I-TIM

In [21]:
gts = []
preds = []
for i in range(len(generator_test)):
    if i % 100 == 0:
      print(f"{i} / {len(generator_test)}")
    try:
      gt = generator_test[i][1]
    except:
      print(i)
      continue

    gt = generator_test[i][1]
    gt = gt.flatten()
    gt = rev_func(gt, generator_test.tag2idx)
    gt = list(filter(lambda x: x != data_generators.PADDING, gt))

    pred = model.predict(generator_test[i][0])[0]
    pred = pred.flatten()
    pred = rev_func(pred, generator_test.tag2idx)
    pred = pred[:len(gt)]
    
    if len(gt) > len(pred):
        gt = gt[:len(pred)]

    gts.extend(gt)
    preds.extend(pred)

    assert len(gts) == len(preds), print(len(gt), len(pred))

gts = np.array(gts).flatten()
preds = np.array(preds).flatten()

weighted_f1 = f1_score(gts, preds, average='weighted')
macro_f1 = f1_score(gts, preds, average='macro')
micro_f1 = f1_score(gts, preds, average='micro')

print('weighted: ', weighted_f1)
print('macro: ', macro_f1)
print('micro: ', micro_f1)

0 / 1582
100 / 1582
200 / 1582
300 / 1582
400 / 1582
500 / 1582
600 / 1582
700 / 1582
800 / 1582
900 / 1582
1000 / 1582
1100 / 1582
1200 / 1582
1300 / 1582
1400 / 1582
1500 / 1582
weighted:  0.8712369815443793
macro:  0.4966734391382058
micro:  0.8818661758976908
